(C:ssp-traitement)=
# Traitement des signaux bruités

## Détection de signaux

Lorsque l'on parle de « détecter un signal », cela signifie répondre à la question : « le signal est-il présent dans le signal observé $y$ ? ».
Dans cette section, on suppose toujours que le bruit est blanc, gaussien et additif :
par conséquent, le signal observé $y$ est la somme du signal recherché $x$ et d'un bruit $b$.
Différentes techniques de détection existent, en fonction des connaissances que l'on a sur le signal recherché $x$.


### Détection de signaux périodiques inconnus de période inconnue

Dans le cas où l'on cherche dans l'observation un signal $x$ périodique, mais qu'on ne connait ni sa période ni sa forme,
alors l'autocorrélation de $y$ permet de déterminer si le signal est présent et quelle est sa période.
On rappelle que l'autocorrélation revient à mesurer la ressemblance d'un signal avec une version plus ou moins décalée de lui-même.

Ainsi, l'autocorrélation de l'observation $y = x + b$ est :

$$
R_y[m] &= \sum_{n=-\infty}^{+\infty} y[n+m] y[n] \\
       &= \sum_{n=-\infty}^{+\infty} (x[n+m]+b[n+m]) (x[n]+b[n]) \\
       &= R_{x}[m] + R_{xb}[m] + R_{bx}[m] + R_{b}[m]
$$

où :
* $R_{x}$ est l'autocorrélation de $x$, et est périodique de même période que $x$ ;
* $R_{xb}$ et $R_{bx}$ sont les intercorrélations entre $x$ et $b$ ; elles sont nulles puisque les deux signaux sont décorrélés ;
* $R_{b}$ est l'autocorrélation du bruit $b$ et elle est nulle puisque le bruit est blanc (ses échantillons sont décorrélés).

Finalement, l'autocorrélation $R_y$ est égale à l'autocorrélation de $x$.
Si $x$ est bien présent dans l'observation, alors $R_y$ est périodique et sa période est égale à la période de $x$.

**figure**


### Détection de signaux périodiques inconnus de période connue

Ce cas est plus favorable que le précédent car on dispose d'une information supplémentaire.
On cherche à déterminer si l'observation $y$ correspond au signal périodique $x$ (de période $T$) bruité par un bruit blanc $b$.
Le principe de détection consiste à effectuer l'intercorrélation de $y$ avec un signal auxiliaire $z$ périodique et de période $T$
(par exemple une simple sinusoïde).

En effet, l'intercorrélation de  $y$ et $z$ est égale à :

$$
R_{yz}[m] &= \sum_{n=-\infty}^{+\infty} y[n+m] z[n] \\
       &= \sum_{n=-\infty}^{+\infty} (x[n+m]+b[n+m]) z[n] \\
       &= R_{xz}[m] + R_{bz}[m].
$$

$R_{xz}$ est périodique puisque $x$ et $z$ sont de même période,
alors que $R_{bz}$ est nulle puisque $b$ et $z$ sont décorrélés.
Ainsi, $R_{yz}$ sera périodique de période $T$ si le signal $x$ est présent.

**figure**


### Détection d'un motif

Un « motif » est un signal déterministe connu et de durée finie.
La technique suivante permet de déterminer si le motif est présent dans l'observation $y$,
et si oui à obtenir son instant d'apparition.
L'observation est donc modélisée :

$$
y[n] = x[n-k] + b[n]
$$

où $k$ est l'instant d'apparition du motif $x$.

On peut déterminer l'instant $k$ où le motif apparaît en calculant l'intercorrélation entre $y$ et $X$.

Comme l'intercorrélation entre $y$ et $x$ est équivalente à la convolution entre $y$ et le motif $x$ retourné dans le temps ($x[-n]$),
cette technique est un filtrage, qu'on appelle « filtrage adapté » (_matched filter_).
On peut montrer qu'elle est la solution optimale pour maximiser le RSB dans le cadre d'un bruit additif.
Elle est notamment utilisé dans des applications radar ou en communications numériques.

**figure**

## Débruitage de signaux

<!-- Débrutées différents détectés  -->

### Débruitage d'un signal unique

<!-- On n'observe : 

Pour réduire le bruit dans le signal (réduire la puissance donc là variance du bruit) on peut remplacer chaque échantillon YN par la moyenne de l échantillon autour de n : 

Ou n un perd 

On peut montrer que cette technique peut s'écrire sous la forme d'une convolution : c'est donc un filtrage appelé filtre moyenne EUR point la réponse impulsionnelle du filtre et une porte de longueur M point là TF t'aider de cette réponse impulsionnelle et ainsi Nice cardinale filtre passe bas 
 -->

### Débruitage de plusieurs réalisations d'un même signal

<!-- ceux de ligne dans le cas où on dispose de I mesure d'un même signal x : 

Et que le bruit et s'en est centré () et de variance Sigma bakaré () alors la moyenne des y qui permet d'augmenter le RSB 

La puissance du bruit est divisée par qui 
 -->

## Approximation par moindres carrés

<!-- Gosse 1795 

On dispose de données y brut est tel que 

Ou F est une fonction paramétrique (sinusoïde polynôme) connu 

FN est la valeur de F 1N 

Titre est un ensemble de paramètres inconnu tête à sous de ligne le bruit bébé et de moyenne nulle 

Exemple 

Polynôme de degré 2 sauts de ligne exponentielle 

L'objectif est de retrouver ta point il est logique de choseaisir téta comme le paramètre qui minimise la somme des différences au carré 

La détermination d'état lorsque effet linéaire en état et simple exemple parabole point mais c'est beaucoup plus compliqué lorsque F n'est pas linéaire (coefficient éteint dans le exponentielle) dans le cas où la fonction F est linéaire son état il existe une expression explicite de la solution FTK égal 

Il faut faire les calculs avec la représentation vectorielle 

  -->